### 1. Import needed packages for pricing topic analysis

In [12]:
import pandas as pd
from collections import Counter

### 2. Read merged CSV file

In [13]:
df = pd.read_csv('clean_data_+_data_pricing.csv')

### 3. Define price set for "Pricing"-topic and look for matches in each review

In [14]:
price_set = {'paywall', 'buy', 'subscription', 'pay', 'payment', 'cost', 'expensive', 'cheap', 'price', 'price', 'pricing', 'purchase', 'fee'}
matches = []
for x in df["content_nonstop"]:
    if price_set.isdisjoint(set(x.split(' '))):
        matches.append(0)
    else:
        matches.append(1)
df = df.assign(contains_pricing=pd.Series(matches))

### 4. Create dataframe for reviews including "Pricing"-topic

In [15]:
df_pricing = df.loc[(df['contains_pricing'] == 1)]

### 5. Create dataframe grouped by sentiment for counts of all reviews and counts of reviews including "Pricing"-topic

counts grouped by sentiment and 1 other column (for example "test_period")

In [16]:
basic = df.groupby('test_period')['sentiment'].value_counts().reset_index(name='counts total')
pricing = df_pricing.groupby('test_period')['sentiment'].value_counts().reset_index(name='counts pricing')

counts grouped by sentiment and 2 other columns (for example "freemium" and "subscription_flatrate")

In [17]:
#basic = df.groupby(['freemium', 'subscription_flatrate'])['sentiment'].value_counts().reset_index(name='counts total')
#pricing = df_pricing.groupby(['freemium', 'subscription_flatrate'])['sentiment'].value_counts().reset_index(name='counts pricing')

### 6. Merge dataframes by 'sentiment' and value you want to compare (i.e. 'test_period', if app offers test periods or not); add calculating columns for percentage of positive, negative or neutral reviews per group and percentage of reviews including "Pricing"-topics

merge dataframes by sentiment and 1 other column (for example "test_period")

In [ ]:
pricing_counts = pd.merge(basic, pricing,  how='left', left_on=['test_period','sentiment'], right_on = ['test_period','sentiment'])     #merge dataframe basic and pricing

pricing_counts['percentage_pricing'] = (pricing_counts['counts pricing'] / pricing_counts['counts total']) * 100        #add percentage of reviews talking about pricing
pricing_counts.percentage_pricing = pricing_counts.percentage_pricing.round(2)      #round percentage 

pricing_counts['percentage_total'] = 100 * pricing_counts['counts total'] / pricing_counts.groupby('test_period')['counts total'].transform('sum')      #add percentage of 'positive', 'negative' or 'neutral' reviews per group
pricing_counts.percentage_total = pricing_counts.percentage_total.round(2)      #round percentage
move_column = pricing_counts.pop('percentage_total')        #move column 'counts total' to another position
pricing_counts.insert(3, 'percentage_total', move_column)


pricing_counts.sort_values('test_period')       #sort values

merge dataframe by sentiment and 2 other column (for example "freemium" and "subscription_flatrate")

In [19]:
#pricing_counts = pd.merge(basic, pricing,  how='left', left_on=['freemium', 'subscription_flatrate', 'sentiment'], right_on = ['freemium', 'subscription_flatrate', 'sentiment'])     #merge dataframe basic and pricing

#pricing_counts['percentage_pricing'] = (pricing_counts['counts pricing'] / pricing_counts['counts total']) * 100        #add percentage of reviews talking about pricing
#pricing_counts.percentage_pricing = pricing_counts.percentage_pricing.round(2)      #round percentage 

#pricing_counts['percentage_total'] = 100 * pricing_counts['counts total'] / pricing_counts.groupby(['freemium', 'subscription_flatrate'])['counts total'].transform('sum')      #add percentage of 'positive', 'negative' or 'neutral' reviews per group
#pricing_counts.percentage_total = pricing_counts.percentage_total.round(2)      #round percentage
#move_column = pricing_counts.pop('percentage_total')        #move column 'counts total' to another position
#pricing_counts.insert(4, 'percentage_total', move_column)


#pricing_counts.sort_values('freemium')


### 7. Export dataframe as CSV

In [20]:
pricing_counts.to_csv('table.csv')